<a href="https://colab.research.google.com/github/azadranjith/kaggle-competitions/blob/main/NLP_TRANSFORMERS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#NLP

  DEEP LEARNING IS DRAMATICALLY IMPROVED IN THE LAST COUPLE OF YEARS IN PROCESSING NATURAL LANGUAGE

  THE MOST USEFULL APPLICATION OF NLP IS CLASSIFYING DOCUMENTS
  
  THIS CAN BE USED FOR 
    
    1.Sentiment analysis
    2.Auther identification
    3.Organizing documents
    4.Legal discovery

🇨 🇴 🇲 🇵 🇪 🇹 🇮 🇹 🇮 🇴 🇳
### US PATENT PHRASE MATCH TO MATCH


comparing 2 words in a short pharases and scoring them based on whether they're similar or not

score 1 considerd to be two inputs have identical meaning

score 0 they have totally different meaning

score 0.5 describes they are  have some what similar not identical

there are scores in between these 0 and 1

**we can turn this problem into a classification problem**

by representing the problem like this 

      "TEXT1: abatement; TEXT2: eliminating process" ...chose a category of meaning similarity: "Different; Similar; Identical".

In [ ]:
from pathlib import Path


this competition uses CSV files. For opening, manipulating, and viewing CSV files, it's generally best to use the Pandas library,

In [ ]:
path = Path('/content/drive/MyDrive/dataset')
path

PosixPath('/content/drive/MyDrive/dataset')

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv(path/'train.csv')
df

,id,anchor,target,context,score
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75
2,36d72442aefd8232,abatement,active catalyst,A47,0.25
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00
...,...,...,...,...,...
36468,8e1386cbefd7f245,wood article,wooden article,B44,1.00
36469,42d9e032d1cd3242,wood article,wooden box,B44,0.50
36470,208654ccb9e14fa3,wood article,wooden handle,B44,0.50
36471,756ec035e694722b,wood article,wooden material,B44,0.75


In [ ]:
df.describe(include='object')#if we add object in describe method it will describe all the non numeric fields

#top row shows the most repeating one,and freq of that in next row

,id,anchor,target,context
count,36473,36473,36473,36473
unique,36473,733,29340,106
top,37d61fd2272659b1,component composite coating,composition,H01
freq,1,152,24,2186


there are 733 unique anchors and nearly 30000 unique target and 106 context 

context are common we can see that H01 apprearing 2186 times





**WE CREATE A SINGLE STRING**

**we need input data as text1: text2: as in one column**


In [ ]:
#adding new colum to table by combining 2 colums
df['input'] ='TEXT1: ' + df.context + '; TEXT2: ' + df.target + '; ANC1: ' + df.anchor
df.head(5)

,id,anchor,target,context,score,input
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50,TEXT1: A47; TEXT2: abatement of pollution; ANC...
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75,TEXT1: A47; TEXT2: act of abating; ANC1: abate...
2,36d72442aefd8232,abatement,active catalyst,A47,0.25,TEXT1: A47; TEXT2: active catalyst; ANC1: abat...
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50,TEXT1: A47; TEXT2: eliminating process; ANC1: ...
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00,TEXT1: A47; TEXT2: forest region; ANC1: abatement


now we got documents to do NLP  ie the input colum

but as we know nn only takes numbers

  STEP 1. Split each of these into TOKENZ (tokens are like spliting into words (like words 😸))
  
  STEP 2. we are going to get list of all unique TOKENZ (vocabulary) and every unique words is going to get a number

  the bigger the vocabulary the bigger the memory going to be used 

  the more data we will needed to train

  so we dont need our vocabulary to be big

In [ ]:
#here we are using hugging faces transformers lib
!pip install transformers

In [ ]:
!pip install datasets

we are turning our dataframe into huggingface datasets Dataset 

In [ ]:
from datasets import Dataset,DatasetDict

In [ ]:
ds = Dataset.from_pandas(df)

In [ ]:
ds

Dataset({
    features: ['id', 'anchor', 'target', 'context', 'score', 'input'],
    num_rows: 36473
})

we need to pass text to the model

but the problem is nueral network expects numbers as inputs

so we need to   
1. tokenize

    split each text into word-like 
2. numericalize 

    turn each word into number based on position of the word in vocabulary


    while tokenizing there is a lot of little decision to be made  
    you don't have to make them

    whatever pretrained model we use that people who made that pretrained model made some decisions we have to do exactly the same thing 

    OTHERWISE WE ENDUP DIFFERENT VOCAB TO THEM 

    that will mess up everything 
there  for before tokenizing we needed to decide what model we are using 

before to

In [ ]:
model_nm= 'microsoft/deberta-v3-small'

now we picked the model only to tokenize in same way 

then we tell to tokenize the way the model did we use 

    AutoTokenizer

    just a dict for which model use which tokenizer



In [ ]:
import transformers

In [ ]:
!pip install sentencepiece
#if any error occures restart the kernel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 8.3 MB/s 


In [ ]:
import sentencepiece


SentencePiece is an unsupervised text tokenizer and detokenizer mainly for Neural Network-based text generation systems where the vocabulary size is predetermined prior to the neural model training. 

[SentencePiece](https://github.com/google/sentencepiece)

In [ ]:
from transformers import AutoModelForSequenceClassification,AutoTokenizer

tokz = AutoTokenizer.from_pretrained(model_nm)#download vocab and the details about how this model tokenize
                                              

In [ ]:
tokz.tokenize(" what the fuck")

['▁what', '▁the', '▁fuck']

In [ ]:
tokz.tokenize(" vennilave vennilave vinaythandivaruvaya")

['▁ven',
 'nil',
 'ave',
 '▁ven',
 'nil',
 'ave',
 '▁vin',
 'ay',
 'th',
 'and',
 'ivar',
 'uva',
 'ya']

each of these kind of words will be passed to the vocabulary and turns into a number

In [ ]:
# now we write a function to tokenize

def tok_func(x):  
  return tokz(x['input'])

tok_ds = tok_func(ds)
we can run it like this 

to run this quickly paralell on every row use map

In [ ]:
tok_ds = ds.map(tok_func, batched=True)
tok_ds

  0%|          | 0/37 [00:00<?, ?ba/s]

Dataset({
    features: ['id', 'anchor', 'target', 'context', 'score', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 36473
})

In [ ]:
tok_ds[0]

{'id': '37d61fd2272659b1',
 'anchor': 'abatement',
 'target': 'abatement of pollution',
 'context': 'A47',
 'score': 0.5,
 'input': 'TEXT1: A47; TEXT2: abatement of pollution; ANC1: abatement',
 'input_ids': [1,
  54453,
  435,
  294,
  336,
  5753,
  346,
  54453,
  445,
  294,
  47284,
  265,
  6435,
  346,
  23702,
  435,
  294,
  47284,
  2],
 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

this numbers in input_ids are the positions of tokenz  in the vocabulary (of this string)


      now we successfully turned text into numbers
      

In [ ]:
tokz.vocab['of'] #position of 'of' in vocab

1580

In [ ]:
len(tokz.vocab)

128001

Finally, we need to prepare our labels. Transformers always assumes that your labels has the column name labels, but in our dataset it's currently score. Therefore, we need to rename it:

In [ ]:
tok_ds = tok_ds.rename_columns({'score' : 'labels'})

In [ ]:
tok_ds

Dataset({
    features: ['id', 'anchor', 'target', 'context', 'labels', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 36473
})

### CREATING A VALIDATION SET

Transformer create a DatasetDict to store your train and validation set

In [ ]:
dds = tok_ds.train_test_split(0.25, seed = 42)
#25% data for validation

In [ ]:
dds
# here the validation set is called the test set
# this way of taking validation set is no good

DatasetDict({
    train: Dataset({
        features: ['id', 'anchor', 'target', 'context', 'labels', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 27354
    })
    test: Dataset({
        features: ['id', 'anchor', 'target', 'context', 'labels', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 9119
    })
})

Although sklearn offers a train_test_split method, this method takes a random subset of the data, which is a poor choice for many real-world problems.

    One of the most likely culprits for this disconnect between results in development vs results in production 
    is a poorly chosen validation set (or even worse, no validation set at all). Depending on the nature of your data

Dr. Rachel Thomas    [good validatioin set](https://https://www.fast.ai/2017/11/13/validation-sets/)

### CREATING A TEST SET

In [ ]:
eval_df = pd.read_csv(path/'test.csv')

In [ ]:
eval_df.describe()

,id,anchor,target,context
count,36,36,36,36
unique,36,34,36,29
top,4112d61851461f60,el display,inorganic photoconductor drum,G02
freq,1,2,1,3


In [ ]:
eval_df['input'] = 'TEXT1: ' + eval_df.context + '; TEXT2: ' + eval_df.target + '; ANC1: ' + eval_df.anchor
#created a column for input


eval_ds = Dataset.from_pandas(eval_df).map(tok_func, batched=True)
#converted dataframe to huggingface dataset and tokenized and numericalized

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
eval_ds[1]

### CHOOSING METRICS

For this data submissions are evaluated on the Pearson correlation coefficient between the predicted and actual similarity scores.


        Transformers expects metrics to be returned as a dict, since that way the trainer knows what label to use

In [ ]:
import numpy as np

In [ ]:
def corr(x,y): return np.corrcoef(x,y)[0][1]

In [ ]:
def corr_d(eval_pred):  return {'pearson':corr(*eval_pred)}

### TRAINING THE MODEL

To train a model in Transformers we'll need this:

In [ ]:
from transformers import TrainingArguments,Trainer

In [ ]:
bs = 128 #batch size acco to our GPU

epochs = 4

In [ ]:
# find the right learning rate by trial and error
#find the largest one which doesn't spoil accuracy
lr = 8e-5

Transformers uses the TrainingArguments class to set up arguments

In [ ]:
args = TrainingArguments('outputs', learning_rate=lr, warmup_ratio=0.1, lr_scheduler_type='cosine', fp16=True,
    evaluation_strategy="epoch", per_device_train_batch_size=bs, per_device_eval_batch_size=bs*2,
    num_train_epochs=epochs, weight_decay=0.01, report_to='none')

In [ ]:
#create model
model = AutoModelForSequenceClassification.from_pretrained(model_nm, num_labels=1)


Some weights of the model checkpoint at microsoft/deberta-v3-small were not used when initializing DebertaV2ForSequenceClassification: ['lm_predictions.lm_head.bias', 'mask_predictions.dense.bias', 'mask_predictions.dense.weight', 'mask_predictions.LayerNorm.weight', 'mask_predictions.classifier.weight', 'lm_predictions.lm_head.LayerNorm.weight', 'mask_predictions.LayerNorm.bias', 'lm_predictions.lm_head.LayerNorm.bias', 'lm_predictions.lm_head.dense.weight', 'mask_predictions.classifier.bias', 'lm_predictions.lm_head.dense.bias']
- This IS expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from 

In [ ]:
#create trainer .trainer combines data and model
trainer = Trainer(model, args, train_dataset=dds['train'], eval_dataset=dds['test'],tokenizer=tokz, compute_metrics=corr_d) #here test is validation set

Using cuda_amp half precision backend


In [ ]:
#train

trainer.train()

The following columns in the training set don't have a corresponding argument in `DebertaV2ForSequenceClassification.forward` and have been ignored: context, anchor, target, id, input. If context, anchor, target, id, input are not expected by `DebertaV2ForSequenceClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 27354
  Num Epochs = 4
  Instantaneous batch size per device = 128
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Gradient Accumulation steps = 1
  Total optimization steps = 856


Epoch,Training Loss,Validation Loss,Pearson
1,No log,0.023741,0.824550
2,No log,0.022316,0.831951
3,0.015000,0.022614,0.834194
4,0.015000,0.023176,0.835080


The following columns in the evaluation set don't have a corresponding argument in `DebertaV2ForSequenceClassification.forward` and have been ignored: context, anchor, target, id, input. If context, anchor, target, id, input are not expected by `DebertaV2ForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 9119
  Batch size = 256


Epoch,Training Loss,Validation Loss


The following columns in the evaluation set don't have a corresponding argument in `DebertaV2ForSequenceClassification.forward` and have been ignored: context, anchor, target, id, input. If context, anchor, target, id, input are not expected by `DebertaV2ForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 9119
  Batch size = 256
Saving model checkpoint to outputs/checkpoint-500
Configuration saved in outputs/checkpoint-500/config.json
Model weights saved in outputs/checkpoint-500/pytorch_model.bin
tokenizer config file saved in outputs/checkpoint-500/tokenizer_config.json
Special tokens file saved in outputs/checkpoint-500/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `DebertaV2ForSequenceClassification.forward` and have been ignored: context, anchor, target, id, input. If context, anchor, target, id, input are not expected by `DebertaV2ForSequenceClassifica

TrainOutput(global_step=856, training_loss=0.012929757064748033, metrics={'train_runtime': 217.8705, 'train_samples_per_second': 502.207, 'train_steps_per_second': 3.929, 'total_flos': 894954797309700.0, 'train_loss': 0.012929757064748033, 'epoch': 4.0})

In [ ]:
preds = trainer.predict(eval_ds).predictions.astype(float)
preds = np.clip(preds,0,1)
preds

The following columns in the test set don't have a corresponding argument in `DebertaV2ForSequenceClassification.forward` and have been ignored: context, anchor, target, id, input. If context, anchor, target, id, input are not expected by `DebertaV2ForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 36
  Batch size = 256


array([[0.44946289],
       [0.60009766],
       [0.57275391],
       [0.31347656],
       [0.        ],
       [0.52246094],
       [0.55712891],
       [0.        ],
       [0.2668457 ],
       [1.        ],
       [0.27636719],
       [0.25537109],
       [0.75390625],
       [0.98291016],
       [0.77099609],
       [0.53417969],
       [0.3605957 ],
       [0.        ],
       [0.6796875 ],
       [0.37451172],
       [0.56689453],
       [0.24536133],
       [0.11340332],
       [0.24890137],
       [0.61181641],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.67138672],
       [0.32788086],
       [0.        ],
       [0.75927734],
       [0.52783203],
       [0.47241211],
       [0.23205566]])

In [ ]:
j = 0
for i in eval_ds['id']:
  pred = preds[j]
  print(i,pred[0])
  j+=1


4112d61851461f60 0.449462890625
09e418c93a776564 0.60009765625
36baf228038e314b 0.57275390625
1f37ead645e7f0c8 0.3134765625
71a5b6ad068d531f 0.0
474c874d0c07bd21 0.5224609375
442c114ed5c4e3c9 0.55712890625
b8ae62ea5e1d8bdb 0.0
faaddaf8fcba8a3f 0.266845703125
ae0262c02566d2ce 1.0
a8808e31641e856d 0.2763671875
16ae4b99d3601e60 0.25537109375
25c555ca3d5a2092 0.75390625
5203a36c501f1b7c 0.98291015625
b9fdc772bb8fd61c 0.77099609375
7aa5908a77a7ec24 0.5341796875
d19ef3979396d47e 0.360595703125
fd83613b7843f5e1 0.0
2a619016908bfa45 0.6796875
733979d75f59770d 0.37451171875
6546846df17f9800 0.56689453125
3ff0e7a35015be69 0.245361328125
12ca31f018a2e2b9 0.1134033203125
03ba802ed4029e4d 0.2489013671875
c404f8b378cbb008 0.61181640625
78243984c02a72e4 0.0
de51114bc0faec3e 0.0
7e3aff857f056bf9 0.0
26c3c6dc6174b589 0.0
b892011ab2e2cabc 0.67138671875
8247ff562ca185cc 0.327880859375
c057aecbba832387 0.0
9f2279ce667b21dc 0.75927734375
b9ea2b06a878df6f 0.52783203125
79795133c30ef097 0.472412109375
25522e

In [ ]:
rows = []
j = 0
for i in eval_ds['id']:
  
  x = i
  preds = trainer.predict(eval_ds).predictions.astype(float)
  preds = np.clip(preds,0,1)
  pred = preds[j]
  row = [x,pred[0]]
  rows.append(row)
  j += 1

In [ ]:
import csv

In [ ]:
fields = ['id','score']
file_name = 'submission.csv'
with open(file_name,'w') as csvfile:
  csvwriter = csv.writer(csvfile)

  csvwriter.writerow(fields)
  csvwriter.writerows(rows)